# Documentação Técnica: Extração de Dados de Anatomia Patológica - Fleury

## Objetivo Principal
**Este notebook realiza a extração e classificação automática de informações clínicas relevantes em laudos de anatomia patológica de mama do Fleury.** Utilizando técnicas de processamento de linguagem natural (NLP) com modelos de linguagem avançados (LLMs), o notebook identifica descritores de malignidade, graus histológicos e nucleares, formação de túbulos, índices mitóticos e tipos histológicos em laudos médicos de anatomia patológica mamária.

## Tecnologias Utilizadas
- **OpenAI/Databricks LLM API**: Para processamento e extração de informações dos textos dos laudos
- **PySpark**: Framework principal para processamento distribuído de dados
- **MLflow**: Para registro de métricas, experimentos e monitoramento dos resultados de extração
- **Delta Lake**: Sistema de armazenamento para tabelas de destino
- **Pandas**: Manipulação de dataframes para processamento local
- **Expressões Regulares (re)**: Validação e extração de padrões específicos nos textos dos laudos
- **Octoops**: Monitoramento e alertas de falhas

## Fluxo de Trabalho/Etapas Principais
1. **Definição dos Parâmetros**: Configuração de tabelas, filtros e critérios de extração
2. **Consulta de Dados**: Extração de laudos recentes da base de dados do Fleury
3. **Processamento via LLM**:
   - Definição do prompt especializado para extração de informações
   - Processamento batch dos laudos via API de LLM
   - Extração estruturada das informações relevantes
4. **Validação dos Resultados**:
   - Comparação entre extração via regex e via LLM
   - Cálculo de métricas de precisão
   - Registro de métricas no MLflow
5. **Persistência dos Dados**: Salvamento dos dados processados em tabela Delta

## Dados Envolvidos
- **Fonte**: Tabela `refined.saude_preventiva.fleury_laudos`
- **Filtros**:
  - Linha de cuidado: "mama"
  - Sexo: "F" (feminino)
  - Siglas de exame: "ANATPATP", "CTPUNC", "FISHHER"
  - Laudos contendo "Topografia: mama"
- **Tabela de Destino**: `refined.saude_preventiva.fleury_laudos_mama_anatomia_patologica_v2`
- **Informações Extraídas**:
  - **Descritores de Malignidade**: carcinoma, invasivo, invasor, sarcoma, metástase, metastático, maligno, maligna, cdi, cli, cdis
  - **Grau Histológico**: 1, 2 ou 3
  - **Grau Nuclear**: 1, 2 ou 3
  - **Formação de Túbulos**: 1, 2 ou 3
  - **Índice Mitótico**: valor numérico
  - **Tipo Histológico**: classificação específica do tipo de carcinoma

## Resultados/Saídas Esperadas
- DataFrame enriquecido com os campos extraídos dos laudos
- Registro de métricas de qualidade da extração via MLflow
- Dados persistidos na tabela Delta `refined.saude_preventiva.fleury_laudos_mama_anatomia_patologica_v2`
- Alertas via Sentinel (Octoops) em caso de falhas ou ausência de dados

## Pré-requisitos
- Ambiente Databricks configurado
- Acesso ao endpoint LLM Databricks (`databricks-llama-4-maverick` ou `teste-maverick`)
- Permissões de acesso às tabelas de origem e destino
- Bibliotecas instaladas: openai, mlflow, pandas, tqdm, databricks-feature-store, octoops

## Considerações Importantes/Observações
- A extração via LLM é comparada com uma extração via regex (considerada pseudo-gold) para validação
- O threshold definido para aceitação da qualidade da extração é de 80%
- O modelo está otimizado para identificar termos específicos de malignidade e classificações dentro do contexto oncológico mamário
- Os resultados são registrados no experimento MLflow `/Shared/saude_preventiva_mama/experiments_fleury_anatomopatologico`
- A persistência utiliza estratégia de merge (upsert) para garantir a atualização de registros já existentes

# Extração de dados - Anatomo Patologico
Serão analisados os descritores de malignidade.
**Descritores de MALIGNIDADE:**
- carcinoma
- invasivo
- invasor
- sarcoma
- metástase
- metastático
- maligno
- maligna
- cdi, cli, cdis

Outros labels a serem extraídos:

- **Grau histológico:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"grau histológico"**.

- **Grau nuclear:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"grau nuclear"**.

- **Formação de túbulos:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"formação de túbulos"**.

- **Índice mitótico:** será sempre um algarismo 1, 2 ou 3 (apenas três categorias). Para encontrar, basta procurar o primeiro algarismo numérico após o termo **"mm2"**. Nesse caso, é melhor procurar o termo **"mm2"** ao invés de **"índice mitótico"**.

- **Labels de tipos histológicos:**
  - Carcinoma de mama ductal invasivo (CDI)/SOE
  - Carcinoma de mama ductal in situ
  - Carcinoma de mama lobular invasivo
  - Carcinoma de mama lobular
  - Carcinoma de mama papilífero
  - Carcinoma de mama metaplásico
  - Carcinoma de mama mucinoso
  - Carcinoma de mama tubular
  - Carcinoma de mama cístico adenoide
  - Carcinoma de mama medular
  - Carcinoma de mama micropapilar
  - Carcinoma de mama misto (ductal e lobular) invasivo

### Instalação de pacotes necessários
**Objetivo da Célula:** Instalar as bibliotecas necessárias para executar o notebook.

**Pacotes Instalados:**
- **openai**: Cliente para comunicação com a API OpenAI/Databricks LLM
- **databricks-feature-store**: Biblioteca para interagir com o Feature Store do Databricks
- **octoops**: Biblioteca para monitoramento e alertas

A instalação é feita usando o comando mágico `%pip`, que é específico para ambientes Jupyter/Databricks. O parâmetro `-q` (quiet) é usado em algumas instalações para reduzir a verbosidade da saída.

In [0]:
def salvar_excel(df, nome_arquivo):
    """ 
    Salva um DataFrame em um arquivo Excel.

    Args:
        df (pandas.DataFrame): DataFrame a ser salvo.
        nome_arquivo (str): Nome do arquivo Excel.
    """
    %pip install openpyxl
    import pandas as pd

    # After installing, save the DataFrame to Excel again
    df.to_excel(nome_arquivo, index=False)

In [0]:
%pip install openai
# %pip install tqdm -q
# %pip install pandarallel -q
%pip install databricks-feature-store -q
%pip install octoops

### Reinicialização do ambiente Python
**Objetivo da Célula:** Reiniciar o kernel Python para garantir que as bibliotecas recém-instaladas sejam carregadas corretamente.

Esta célula executa o comando `dbutils.library.restartPython()`, que é específico do ambiente Databricks. Este comando reinicia o interpretador Python, garantindo que todas as bibliotecas instaladas na célula anterior sejam devidamente carregadas no ambiente de execução. Isso é necessário porque, em ambientes Jupyter/Databricks, as bibliotecas instaladas durante a execução do notebook só ficam disponíveis após a reinicialização do kernel.

In [0]:
dbutils.library.restartPython()

### Importação de bibliotecas e configuração do ambiente
**Objetivo da Célula:** Importar todas as bibliotecas necessárias para o processamento de dados e desativar exibições automáticas do MLflow.

**Dependências:**
- Bibliotecas instaladas nas células anteriores

**Bibliotecas Importadas:**
- **re**: Para manipulação de expressões regulares
- **os, sys**: Manipulação do sistema e ambiente
- **json, time**: Processamento de JSON e controle de tempo
- **warnings**: Controle de mensagens de aviso
- **mlflow**: Rastreamento de experimentos e métricas
- **tqdm**: Barras de progresso para processos iterativos
- **pandas, numpy**: Manipulação e análise de dados
- **typing**: Anotações de tipo para melhor documentação do código
- **openai**: Cliente para API de LLM
- **dateutil.relativedelta**: Cálculos avançados de datas
- **pyspark.sql**: Componentes do Spark SQL
- **databricks.feature_store**: Cliente para Feature Store do Databricks

**Configurações Realizadas:**
- Desativa a exibição automática do MLflow no notebook com `mlflow.tracing.disable_notebook_display()`

**Variáveis/Objetos Criados:**
- Não cria variáveis persistentes além das importações

**Saída/Impacto:**
- Prepara o ambiente de execução com todas as dependências necessárias
- Desativa logs automáticos do MLflow que poderiam sobrecarregar a interface do notebook

In [0]:
import re
import os
import sys
import json
import time
import warnings
import mlflow
import pandas as pd
import numpy as np
from typing import List, Any
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, udf
from databricks.feature_store import FeatureStoreClient
from delta.tables import DeltaTable

mlflow.tracing.disable_notebook_display()

### Definição de tabelas e filtros para extração de dados
**Objetivo da Célula:** Configurar as variáveis de tabela de destino e definir os filtros SQL para seleção de dados recentes e relevantes.

**Variáveis/Objetos Criados:**
- `table_anatom`: String com o nome completo da tabela de destino (refined.saude_preventiva.fleury_laudos_mama_anatomia_patologica_v2)
- `where_clause`: String contendo cláusula SQL WHERE para selecionar apenas registros novos (com datestamp maior que o último processado)
- `filtro_extracao`: String contendo filtros para seleção de registros específicos de mama feminina

**Lógica Detalhada:**
- Define a tabela de destino para resultados de anatomia patológica
- Cria uma cláusula WHERE que seleciona apenas registros mais recentes que o último datestamp na tabela de destino (processamento incremental)
- Define filtros para extração que limitam os registros aos laudos:
  - Da linha de cuidado "mama"
  - De pacientes do sexo feminino (UPPER(sexo_cliente) = 'F')
  - Com siglas de exames específicas (ANATPATP, CTPUNC, FISHHER)
  - Contendo a expressão "Topografia: mama" no texto do laudo

**Saída/Impacto:**
- As variáveis definidas serão utilizadas posteriormente na consulta SQL para extrair dados relevantes de laudos de anatomia patológica mamária
- A abordagem de filtro por datestamp garante processamento incremental, evitando reprocessamento de registros já analisados

In [0]:
# filtros de extração
table_anatom = "refined.saude_preventiva.fleury_laudos_mama_anatomia_patologica_v2" 

where_clause = f"""
WHERE
    flr.`_datestamp` >= (
        SELECT MAX(anatom._datestamp)
        FROM {table_anatom} anatom
    )
    """

 
filtro_extracao = """
    WHERE
        linha_cuidado  = 'mama'
        AND UPPER(sexo_cliente) = 'F'
        AND sigla_exame IN ("ANATPATP", "CTPUNC", "FISHHER")
        AND laudo_tratado RLIKE '(?i)Topografia: mama'
    
"""

### Construção e execução da consulta SQL
**Objetivo da Célula:** Criar uma consulta SQL para extrair laudos relevantes e executá-la para obter os dados a serem processados pelo modelo LLM.

**Variáveis/Objetos Criados:**
- `query`: String contendo a consulta SQL completa com placeholders para os filtros definidos anteriormente
- `df_spk`: DataFrame Spark resultante da execução da consulta

**Lógica Detalhada:**
1. Define uma consulta SQL com Common Table Expression (CTE) chamada `base`
2. Seleciona campos relevantes dos laudos: identificadores, datas, informações do exame e o conteúdo do laudo
3. Aplica os filtros definidos anteriormente:
   - Processamento incremental via `where_clause` (registros mais recentes que o último processado)
   - Filtros específicos de mama via `filtro_extracao` (linha de cuidado, sexo, siglas de exame, conteúdo do laudo)
4. Executa a consulta usando `spark.sql()`, formatando a string para incluir os filtros
5. Exibe o resultado via `display(df_spk)`

**Tabelas/Colunas Utilizadas:**
- Tabela: `refined.saude_preventiva.fleury_laudos` (alias `flr`)
- Colunas principais:
  - Identificadores: `id_marca`, `id_unidade`, `id_cliente`, `id_ficha`, `ficha`, `id_item`, `id_subitem`, `id_exame`
  - Datas: `dth_pedido`, `dth_resultado`
  - Dados do exame: `sigla_exame`, `laudo_tratado`, `linha_cuidado`, `sexo_cliente`
  - Metadata: `_datestamp`

**Saída/Impacto:**
- Cria o DataFrame `df_spk` contendo os laudos de anatomia patológica mamária que serão processados
- Exibe o conteúdo do DataFrame na interface do notebook para visualização prévia dos dados

In [0]:
query = f"""
WITH 
base AS (
    SELECT
        flr.id_marca,
        flr.id_unidade,
        flr.id_cliente, 
        flr.id_ficha,
        flr.ficha,
        flr.id_item, 
        flr.id_subitem, 
        flr.id_exame, 
        flr.dth_pedido,
        flr.dth_resultado,
        flr.sigla_exame,
        flr.laudo_tratado,
        flr.linha_cuidado,
        flr.sexo_cliente,
        flr.`_datestamp`
    FROM refined.saude_preventiva.fleury_laudos flr 
    {where_clause} 
     
)
SELECT *
FROM base
{filtro_extracao}
"""
df_spk = spark.sql(query)
display(df_spk)

In [0]:
# Template do prompt para uso com F.format_string
# Use %s como placeholder e escape %% literal
prompt_laudo_template = """A seguir está um laudo médico de mamografia. Se alguma informação não estiver presente no texto, retorne "NÃO INFORMADO". Retorne APENAS um JSON válido, sem blocos de código e sem comentários.

Laudo clínico:
\"\"\"%s\"\"\"

### Critérios de extração:

- **Descritores de malignidade**: retorne uma **lista** com os termos de malignidade encontrados no texto (case-insensitive). Se nenhum for encontrado, retorne lista vazia []. Lista de termos: ["carcinoma", "invasivo", "invasor", "sarcoma", "metástase", "metastático", "maligno", "maligna", "cdi", "cli", "cdis"]

- **Grau histológico**: retorne o valor numérico do grau histológico ou "NÃO INFORMADO".

- **Grau nuclear**: retorne o valor numérico do grau nuclear ou "NÃO INFORMADO".

- **Formação de túbulos**: retorne o valor numérico caso exista formação de túbulos ou "NÃO INFORMADO".

- **Índice mitótico**: retorne o valor numérico do score do índice mitótico que aparece após o mm2 ou "NÃO INFORMADO".

- **Tipo histológico**: identifique e retorne a frase correspondente se algum dos seguintes for mencionado (case-insensitive, variações aceitas):
- Carcinoma de mama ductal invasivo
- Carcinoma de mama ductal in situ
- Carcinoma de mama lobular invasivo
- Carcinoma de mama lobular
- Carcinoma de mama papilífero
- Carcinoma de mama metaplásico
- Carcinoma de mama mucinoso
- Carcinoma de mama tubular
- Carcinoma de mama cístico adenoide
- Carcinoma de mama medular
- Carcinoma de mama micropapilar
- Carcinoma de mama misto (ductal e lobular) invasivo


### Exemplo de saída (JSON válido):
{
  "descritores_malignidade": ["carcinoma", "invasivo"], 
  "grau_histologico": "1", 
  "grau_nuclear": "2", 
  "formacao_tubulos": "2", 
  "indice_mitotico": "1", 
  "tipo_histologico": "Carcinoma de mama ductal invasivo"
}

"""

# Schema para parsear a resposta JSON do LLM
llm_output_schema = StructType(
  [
    StructField("descritores_malignidade", ArrayType(StringType()), True),
    StructField("grau_histologico", StringType(), True),
    StructField("grau_nuclear", StringType(), True),
    StructField("formacao_tubulos", StringType(), True),
    StructField("indice_mitotico", StringType(), True),
    StructField("tipo_histologico", StringType(), True),
  ]
)

### Funções para geração de respostas usando o LLM
**Objetivo da Célula:** Definir funções que gerenciam as chamadas à API do LLM e processam respostas para análise de laudos médicos.

**Funções Definidas:**
1. `generate(descricao_agente:str, laudo:str, llm_client) -> str`: Função principal para enviar um laudo ao modelo e obter resposta
2. `batch_generate(descricao_agente, laudos, llm_client, batch_size=25)`: Função para processar múltiplos laudos em lotes

**Lógica Detalhada da Função `generate`:**
1. Recebe os parâmetros: descrição do papel do LLM, texto do laudo, e o cliente da API
2. Cria o prompt específico para o laudo usando a função `prompt_laudo()`
3. Monta a estrutura de mensagens para a API:
   - Mensagem de sistema definindo o papel do LLM
   - Mensagem do usuário contendo o prompt com o laudo
4. Define parâmetros para a chamada ao modelo:
   - Modelo: "teste-maverick" (endpoint do Databricks)
   - Temperatura: 0 (determinístico)
   - Tokens máximos: 4000
   - Outros parâmetros de controle da geração de texto
5. Implementa mecanismo de retry (3 tentativas) em caso de falha de conexão
6. Retorna o texto da resposta do modelo

**Lógica Detalhada da Função `batch_generate`:**
1. Recebe listas de laudos, descrição do agente, cliente da API e tamanho do lote
2. Inicializa o cliente OpenAI com o token do Databricks
3. Divide os laudos em lotes menores (padrão: 25 laudos por lote)
4. Para cada lote, processa cada laudo individualmente usando a função `generate()`
5. Exibe barra de progresso usando tqdm
6. Retorna lista com todas as respostas do modelo

**Dependências:**
- Função `prompt_laudo()` definida anteriormente
- Variável `DATABRICKS_TOKEN` para autenticação
- Biblioteca tqdm para exibição de progresso
- Cliente OpenAI para comunicação com a API

**Saída/Impacto:**
- As funções definidas serão utilizadas posteriormente para processamento dos laudos extraídos
- O mecanismo de retry aumenta a robustez do sistema a falhas temporárias de rede/API
- O processamento em batch otimiza o uso da API e permite monitoramento de progresso

In [0]:
# Funções generate e batch_generate removidas
# A extração será feita diretamente no Spark DataFrame usando ai_query + F.expr
# Não é mais necessário coletar os dados para pandas nem usar cliente OpenAI

# Nome do endpoint do Foundation Model
ENDPOINT_NAME = "databricks-llama-4-maverick"

### Funções para extração por RegEx e validação das respostas do LLM
**Objetivo da Célula:** Implementar um conjunto de funções que extraem informações dos laudos usando expressões regulares e comparam com as respostas do LLM.

**Funções Definidas:**
1. Funções de extração por RegEx:
   - `extrai_descritores`: Encontra descritores de malignidade no texto
   - `extrai_grau_histologico`: Extrai o grau histológico (1, 2 ou 3)
   - `extrai_grau_nuclear`: Extrai o grau nuclear (1, 2 ou 3)
   - `extrai_formacao_tubulos`: Extrai o valor de formação de túbulos (1, 2 ou 3)
   - `extrai_indice_mitotico`: Extrai o índice mitótico
   - `extrai_tipo_histologico`: Identifica o tipo histológico específico

2. Função de avaliação:
   - `avalia_extracao_sem_ground_truth`: Compara a extração do LLM com a extração por RegEx

**Lógica Detalhada:**
- `extrai_descritores`: Procura por cada termo da lista TERMS no texto usando regex case-insensitive
- `extrai_grau_histologico`, `extrai_grau_nuclear`, `extrai_formacao_tubulos`: Procuram por padrões específicos seguidos de um dígito
- `extrai_indice_mitotico`: Procura por um valor numérico próximo ao termo "mm2" ou "mitoses"
- `extrai_tipo_histologico`: Compara o texto com uma lista predefinida de tipos histológicos
- `avalia_extracao_sem_ground_truth`: 
  1. Gera um pseudo-gold standard usando as funções de extração por RegEx
  2. Compara campo a campo com as extrações do LLM
  3. Retorna um relatório detalhado de concordância

**Constantes Definidas:**
- `TERMS`: Lista de termos associados a malignidade
- `TIPOS`: Lista de padrões de tipos histológicos a serem buscados

**Saída/Impacto:**
- As funções serão utilizadas para avaliar a qualidade das extrações do modelo LLM
- A avaliação usa as extrações por RegEx como pseudo-gold standard
- Os resultados da comparação serão utilizados para métricas e monitoramento do desempenho do modelo

In [0]:
TERMS = ['carcinoma', 'invasivo', 'invasor', 'sarcoma', 
       'metástase', 'metastático', 'maligno', 'maligna', 
       'cdi', 'cli', 'cdis']

def extrai_descritores(txt):
  if txt is None:
      return []
  achados = set()
  for termo in TERMS:
      if re.search(rf"\b{re.escape(termo)}\b", txt, flags=re.IGNORECASE):
          achados.add(termo.lower())
  return sorted(achados)

def extrai_grau_histologico(txt):
  if txt is None:
      return None
  m = re.search(r"grau\s+histol[oó]gico\s*[:\-]?\s*(\d)", txt, flags=re.IGNORECASE)
  if m:
      return int(m.group(1))
  return None

def extrai_grau_nuclear(txt):
  if txt is None:
      return None
  m = re.search(r"grau\s+nuclear\s*[:\-]?\s*(\d)", txt, flags=re.IGNORECASE)
  return int(m.group(1)) if m else None

def extrai_formacao_tubulos(txt):
  if txt is None:
      return None
  m = re.search(r"forma[cç][aã]o\s+de\s+t[uú]bulos\s*[:\-]?\s*(\d)", txt, flags=re.IGNORECASE)
  return int(m.group(1)) if m else None

def extrai_indice_mitotico(txt):
  if txt is None:
      return None
  m = re.search(r"mit[oó]tico\s*[:\-]?\s*(\d+)(?:\s*/\s*\d+)?\s*(?:mitoses?|mitose)?\s*/?\s*mm2", txt, flags=re.IGNORECASE)
  if m:
      return int(m.group(1))
  return None

TIPOS = [
  "carcinoma de mama ductal invasivo",
  "carcinoma de mama ductal in situ",
  "carcinoma de mama lobular invasivo",
  "carcinoma de mama lobular",
  "carcinoma de mama papilífero",
  "carcinoma de mama metapl[aá]sico",
  "carcinoma de mama mucinoso",
  "carcinoma de mama tubular",
  "carcinoma de mama c[ií]stico adenoide",
  "carcinoma de mama medular",
  "carcinoma de mama micropapilar",
  "carcinoma de mama misto (ductal e lobular) invasivo"
]

def extrai_tipo_histologico(txt):
  if txt is None:
      return None
  txt_lower = txt.lower()
  for tipo in TIPOS:
      padrao = tipo.lower()
      if padrao in txt_lower:
          return tipo
  return None

# Registrar as funções como UDFs para uso no Spark
extrai_descritores_udf = udf(extrai_descritores, ArrayType(StringType()))
extrai_grau_histologico_udf = udf(extrai_grau_histologico, IntegerType())
extrai_grau_nuclear_udf = udf(extrai_grau_nuclear, IntegerType())
extrai_formacao_tubulos_udf = udf(extrai_formacao_tubulos, IntegerType())
extrai_indice_mitotico_udf = udf(extrai_indice_mitotico, IntegerType())
extrai_tipo_histologico_udf = udf(extrai_tipo_histologico, StringType())

### Processamento e avaliação dos laudos com o modelo LLM
**Objetivo da Célula:** Processar os laudos extraídos utilizando o modelo LLM, avaliar a qualidade das extrações e preparar os dados para persistência.

**Dependências:**
- DataFrames e funções definidas anteriormente
- Token de autenticação Databricks
- Cliente OpenAI configurado

**Lógica Detalhada:**
1. Verifica se há dados disponíveis para processamento (`df_spk.count() > 0`)
2. Inicializa o cliente OpenAI com o token Databricks e configuração do endpoint
3. Define o contexto do agente LLM como "médico oncologista especialista em laudos de mamografia"
4. Converte o DataFrame Spark para Pandas para processamento local (limitado a 15 registros para teste)
5. Processa os laudos em batch utilizando a função `batch_generate`
6. Limpa e converte as respostas do LLM para formato estruturado utilizando `limpar_e_converter`
7. Avalia a qualidade das extrações comparando com extrações via RegEx
8. Calcula métricas agregadas de precisão

**Variáveis/Objetos Criados:**
- `llm_client`: Cliente para comunicação com a API do LLM
- `descricao_agente`: Descrição do papel do LLM na análise
- `df_local`: DataFrame Pandas para processamento local
- `df_respostas`: DataFrame Spark com as respostas processadas
- `resultados`: Lista de comparações entre extrações via LLM e RegEx
- `json_metricas`: Estatísticas agregadas sobre a qualidade da extração

**Saída/Impacto:**
- Cria um DataFrame enriquecido com as informações extraídas dos laudos
- Exibe o DataFrame processado via `display(df_respostas)`
- Prepara os dados para registro de métricas no MLflow e persistência em tabela Delta

In [0]:
# Verificar e corrigir o filtro should_call_llm
if df_spk.count() > 0:
    # Versão corrigida: usar pattern mais simples sem word boundaries complexos
    termos_escaped = [re.escape(t) for t in TERMS]
    termos_pattern = "(?i)(" + "|".join(termos_escaped) + ")"

    df_test = df_spk.withColumn("should_call_llm", F.expr(f"laudo_tratado RLIKE '{termos_pattern}'"))

    print("Verificando filtro should_call_llm CORRIGIDO:")
    df_test.select("should_call_llm").groupBy("should_call_llm").count().show()

    # Mostrar exemplo de laudo que deveria dar match
    print("\nPrimeiros 100 caracteres de um laudo:")
    df_test.select(F.substring("laudo_tratado", 1, 100).alias("inicio_laudo"), "should_call_llm").show(1, truncate=False)

In [0]:
# Extração usando ai_query diretamente no Spark DataFrame

if df_spk.count() > 0:
    # 1. Opcional: Filtro para reduzir custo (apenas laudos com termos relevantes)
    # Como ai_query funciona bem, você pode usar ou não este filtro
    termos_escaped = [re.escape(t) for t in TERMS]
    termos_pattern = "(?i)(" + "|".join(termos_escaped) + ")"

    df_filtered = df_spk.withColumn("should_call_llm", F.expr(f"laudo_tratado RLIKE '{termos_pattern}'"))

    print(f"Total de laudos: {df_filtered.count()}")
    print("Distribuição do filtro:")
    df_filtered.groupBy("should_call_llm").count().show()

    # 2. Gerar prompt para cada laudo
    df_with_prompt = df_filtered.withColumn(
        "prompt_llm",
        F.format_string(prompt_laudo_template, F.col("laudo_tratado"))
    )

    # 3. Chamar o LLM via ai_query
    # OPÇÃO A: Chamar para todos (sem filtro)
    df_llm_raw = df_with_prompt.withColumn(
        "resp_raw",
        F.expr(f"ai_query('{ENDPOINT_NAME}', prompt_llm)")
    )

    # OPÇÃO B: Chamar apenas para laudos relevantes (com filtro - descomente se preferir)
    # df_llm_raw = df_with_prompt.withColumn(
    #     "resp_raw",
    #     F.when(
    #         F.col("should_call_llm"),
    #         F.expr(f"ai_query('{ENDPOINT_NAME}', prompt_llm)")
    #     ).otherwise(F.lit('{"descritores_malignidade":[], "grau_histologico":"NÃO INFORMADO","grau_nuclear":"NÃO INFORMADO","formacao_tubulos":"NÃO INFORMADO","indice_mitotico":"NÃO INFORMADO","tipo_histologico":"NÃO INFORMADO"}'))
    # )

    # 4. Limpar blocos de código caso o modelo retorne com ```json ou ```python
    df_llm_clean = df_llm_raw.withColumn(
        "resp_clean",
        F.regexp_replace(
            F.regexp_replace(F.col("resp_raw"), "```(?:json|python)?", ""),
            "```", ""
        )
    )

    # 5. Parsear JSON com from_json
    df_llm_parsed = df_llm_clean.withColumn(
        "resp",
        F.from_json(F.trim(F.col("resp_clean")), llm_output_schema)
    )

    # 6. Extrair campos da estrutura JSON parseada e converter numéricos de forma segura
    def cast_int_safe(colname):
        return F.when(
            F.lower(F.col(f"resp.{colname}")).isin("não informado", "nao informado"), 
            F.lit(None)
        ).when(
            F.col(f"resp.{colname}").isNull(),
            F.lit(None)
        ).otherwise(
            F.col(f"resp.{colname}").cast(IntegerType())
        )

    df_llm_final = (
        df_llm_parsed
        .withColumn("descritores_malignidade", F.col("resp.descritores_malignidade"))
        .withColumn("grau_histologico", cast_int_safe("grau_histologico"))
        .withColumn("grau_nuclear", cast_int_safe("grau_nuclear"))
        .withColumn("formacao_tubulos", cast_int_safe("formacao_tubulos"))
        .withColumn("indice_mitotico", cast_int_safe("indice_mitotico"))
        .withColumn("tipo_histologico", F.col("resp.tipo_histologico"))
    )

    # 7. Aplicar heurísticas (pseudo-gold) usando UDFs
    df_with_heuristics = (
        df_llm_final
        .withColumn("heu_descritores", extrai_descritores_udf(F.col("laudo_tratado")))
        .withColumn("heu_grau_histologico", extrai_grau_histologico_udf(F.col("laudo_tratado")))
        .withColumn("heu_grau_nuclear", extrai_grau_nuclear_udf(F.col("laudo_tratado")))
        .withColumn("heu_formacao_tubulos", extrai_formacao_tubulos_udf(F.col("laudo_tratado")))
        .withColumn("heu_indice_mitotico", extrai_indice_mitotico_udf(F.col("laudo_tratado")))
        .withColumn("heu_tipo_histologico", extrai_tipo_histologico_udf(F.col("laudo_tratado")))
    )

    # 8. Comparar LLM vs Heurística campo a campo
    df_comparisons = (
        df_with_heuristics
        .withColumn("acertou_descritores", 
                    F.sort_array(F.coalesce(F.col("descritores_malignidade"), F.array())) == 
                    F.sort_array(F.coalesce(F.col("heu_descritores"), F.array())))
        .withColumn("acertou_grau_histologico", 
                    (F.col("grau_histologico") == F.col("heu_grau_histologico")) | 
                    (F.col("grau_histologico").isNull() & F.col("heu_grau_histologico").isNull()))
        .withColumn("acertou_grau_nuclear", 
                    (F.col("grau_nuclear") == F.col("heu_grau_nuclear")) | 
                    (F.col("grau_nuclear").isNull() & F.col("heu_grau_nuclear").isNull()))
        .withColumn("acertou_formacao_tubulos", 
                    (F.col("formacao_tubulos") == F.col("heu_formacao_tubulos")) | 
                    (F.col("formacao_tubulos").isNull() & F.col("heu_formacao_tubulos").isNull()))
        .withColumn("acertou_indice_mitotico", 
                    (F.col("indice_mitotico") == F.col("heu_indice_mitotico")) | 
                    (F.col("indice_mitotico").isNull() & F.col("heu_indice_mitotico").isNull()))
        .withColumn("acertou_tipo_histologico", 
                    F.col("tipo_histologico") == F.col("heu_tipo_histologico"))
    )

    # DataFrame final com todas as extrações e comparações
    df_respostas = df_comparisons

    # 9. Calcular métricas agregadas
    total_laudos = df_respostas.count()

    def calcular_taxa_acerto(col_name):
        if total_laudos == 0:
            return {"acertos": 0, "total": 0, "taxa_acerto": 0.0}
        taxa = df_respostas.select(F.avg(F.col(col_name).cast("double")).alias("taxa")).collect()[0]["taxa"]
        if taxa is None:
            taxa = 0.0
        acertos = int(taxa * total_laudos)
        return {
            "acertos": acertos,
            "total": total_laudos,
            "taxa_acerto": taxa
        }

    json_metricas = {
        "descritores_malignidade": calcular_taxa_acerto("acertou_descritores"),
        "grau_histologico": calcular_taxa_acerto("acertou_grau_histologico"),
        "grau_nuclear": calcular_taxa_acerto("acertou_grau_nuclear"),
        "formacao_tubulos": calcular_taxa_acerto("acertou_formacao_tubulos"),
        "indice_mitotico": calcular_taxa_acerto("acertou_indice_mitotico"),
        "tipo_histologico": calcular_taxa_acerto("acertou_tipo_histologico"),
    }

    print(f"\n✓ Processamento concluído: {total_laudos} laudos")
    display(df_respostas)

In [0]:
# correção ======================
# Extração usando ai_query via SQL puro (mais confiável no contexto distribuído)

if df_spk.count() > 0:
    # 1. Opcional: Filtro para reduzir custo
    termos_escaped = [re.escape(t) for t in TERMS]
    termos_pattern = "(?i)(" + "|".join(termos_escaped) + ")"

    df_filtered = df_spk.withColumn("should_call_llm", F.expr(f"laudo_tratado RLIKE '{termos_pattern}'"))

    print(f"Total de laudos: {df_filtered.count()}")
    print("Distribuição do filtro:")
    df_filtered.groupBy("should_call_llm").count().show()

    # 2. Gerar prompt usando CONCAT
    prompt_prefix = """A seguir está um laudo médico de mamografia. Se alguma informação não estiver presente no texto, retorne "NÃO INFORMADO". Retorne APENAS um JSON válido, sem blocos de código e sem comentários.

    Laudo clínico:
    \"\"\"
    """

    prompt_suffix = """
    \"\"\"

    ### Critérios de extração:

    - **Descritores de malignidade**: retorne uma **lista** com os termos de malignidade encontrados no texto (case-insensitive). Se nenhum for encontrado, retorne lista vazia []. Lista de termos: ["carcinoma", "invasivo", "invasor", "sarcoma", "metástase", "metastático", "maligno", "maligna", "cdi", "cli", "cdis"]

    - **Grau histológico**: retorne o valor numérico do grau histológico ou "NÃO INFORMADO".

    - **Grau nuclear**: retorne o valor numérico do grau nuclear ou "NÃO INFORMADO".

    - **Formação de túbulos**: retorne o valor numérico do escore caso exista formação de túbulos ou "NÃO INFORMADO".

    - **Índice mitótico**: retorne o valor numérico do escore do índice mitótico que aparece entre parênteses ou "NÃO INFORMADO".

    - **Tipo histológico**: identifique e retorne a frase correspondente se algum dos seguintes for mencionado (case-insensitive): Carcinoma de mama ductal invasivo, Carcinoma de mama ductal in situ, Carcinoma de mama lobular invasivo, Carcinoma de mama lobular, Carcinoma de mama papilífero, Carcinoma de mama metaplásico, Carcinoma de mama mucinoso, Carcinoma de mama tubular, Carcinoma de mama cístico adenoide, Carcinoma de mama medular, Carcinoma de mama micropapilar, Carcinoma de mama misto (ductal e lobular) invasivo, ou "NÃO INFORMADO".

    ### Exemplo de saída (JSON válido):
    {"descritores_malignidade": ["carcinoma", "invasivo"], "grau_histologico": "1", "grau_nuclear": "2", "formacao_tubulos": "2", "indice_mitotico": "1", "tipo_histologico": "Carcinoma de mama ductal invasivo"}
    """

    df_with_prompt = df_filtered.withColumn(
        "prompt_llm",
        F.concat(
            F.lit(prompt_prefix),
            F.col("laudo_tratado"),
            F.lit(prompt_suffix)
        )
    )

    # 3. Registrar como tabela temporária e usar SQL PURO
    df_with_prompt.createOrReplaceTempView("laudos_para_processar")

    # 4. Chamar ai_query via SQL (método mais confiável)
    df_llm_raw = spark.sql(f"""
        SELECT 
            *,
            ai_query('{ENDPOINT_NAME}', prompt_llm) as resp_raw
        FROM laudos_para_processar
    """)

    print("Primeiras 2 respostas do modelo:")
    df_llm_raw.select(F.substring("resp_raw", 1, 200).alias("resposta_inicio")).show(2, truncate=False)

    # 5. Limpar blocos de código
    df_llm_clean = df_llm_raw.withColumn(
        "resp_clean",
        F.regexp_replace(
            F.regexp_replace(F.col("resp_raw"), "```(?:json|python)?", ""),
            "```", ""
        )
    )

    # 6. Parsear JSON
    df_llm_parsed = df_llm_clean.withColumn(
        "resp",
        F.from_json(F.trim(F.col("resp_clean")), llm_output_schema)
    )

    # 7. Extrair campos e converter numéricos
    def cast_int_safe(colname):
        return F.when(
            F.lower(F.col(f"resp.{colname}")).isin("não informado", "nao informado"), 
            F.lit(None)
        ).when(
            F.col(f"resp.{colname}").isNull(),
            F.lit(None)
        ).otherwise(
            F.col(f"resp.{colname}").cast(IntegerType())
        )

    df_llm_final = (
        df_llm_parsed
        .withColumn("descritores_malignidade", F.coalesce(F.col("resp.descritores_malignidade"), F.array()))
        .withColumn("grau_histologico", cast_int_safe("grau_histologico"))
        .withColumn("grau_nuclear", cast_int_safe("grau_nuclear"))
        .withColumn("formacao_tubulos", cast_int_safe("formacao_tubulos"))
        .withColumn("indice_mitotico", cast_int_safe("indice_mitotico"))
        .withColumn("tipo_histologico", F.coalesce(F.col("resp.tipo_histologico"), F.lit("NÃO INFORMADO")))
    )

    # 8. Aplicar heurísticas
    df_with_heuristics = (
        df_llm_final
        .withColumn("heu_descritores", extrai_descritores_udf(F.col("laudo_tratado")))
        .withColumn("heu_grau_histologico", extrai_grau_histologico_udf(F.col("laudo_tratado")))
        .withColumn("heu_grau_nuclear", extrai_grau_nuclear_udf(F.col("laudo_tratado")))
        .withColumn("heu_formacao_tubulos", extrai_formacao_tubulos_udf(F.col("laudo_tratado")))
        .withColumn("heu_indice_mitotico", extrai_indice_mitotico_udf(F.col("laudo_tratado")))
        .withColumn("heu_tipo_histologico", extrai_tipo_histologico_udf(F.col("laudo_tratado")))
    )

    # 9. Comparar - NÃO contar NULL==NULL como acerto
    df_comparisons = (
        df_with_heuristics
        .withColumn("acertou_descritores", 
                    F.sort_array(F.col("descritores_malignidade")) == F.sort_array(F.col("heu_descritores")))
        .withColumn("acertou_grau_histologico", 
                    F.when(F.col("grau_histologico").isNull() | F.col("heu_grau_histologico").isNull(), F.lit(False))
                    .otherwise(F.col("grau_histologico") == F.col("heu_grau_histologico")))
        .withColumn("acertou_grau_nuclear", 
                    F.when(F.col("grau_nuclear").isNull() | F.col("heu_grau_nuclear").isNull(), F.lit(False))
                    .otherwise(F.col("grau_nuclear") == F.col("heu_grau_nuclear")))
        .withColumn("acertou_formacao_tubulos", 
                    F.when(F.col("formacao_tubulos").isNull() | F.col("heu_formacao_tubulos").isNull(), F.lit(False))
                    .otherwise(F.col("formacao_tubulos") == F.col("heu_formacao_tubulos")))
        .withColumn("acertou_indice_mitotico", 
                    F.when(F.col("indice_mitotico").isNull() | F.col("heu_indice_mitotico").isNull(), F.lit(False))
                    .otherwise(F.col("indice_mitotico") == F.col("heu_indice_mitotico")))
        .withColumn("acertou_tipo_histologico",
                    F.when((F.col("tipo_histologico") == "NÃO INFORMADO") | F.col("heu_tipo_histologico").isNull(), F.lit(False))
                    .otherwise(F.col("tipo_histologico") == F.col("heu_tipo_histologico")))
    )

    df_respostas = df_comparisons

    # 10. Calcular métricas
    total_laudos = df_respostas.count()

    def calcular_taxa_acerto(col_name):
        if total_laudos == 0:
            return {"acertos": 0, "total": 0, "taxa_acerto": 0.0}
        taxa = df_respostas.select(F.avg(F.col(col_name).cast("double")).alias("taxa")).collect()[0]["taxa"]
        if taxa is None:
            taxa = 0.0
        acertos = int(taxa * total_laudos)
        return {
            "acertos": acertos,
            "total": total_laudos,
            "taxa_acerto": taxa
        }

    json_metricas = {
        "descritores_malignidade": calcular_taxa_acerto("acertou_descritores"),
        "grau_histologico": calcular_taxa_acerto("acertou_grau_histologico"),
        "grau_nuclear": calcular_taxa_acerto("acertou_grau_nuclear"),
        "formacao_tubulos": calcular_taxa_acerto("acertou_formacao_tubulos"),
        "indice_mitotico": calcular_taxa_acerto("acertou_indice_mitotico"),
        "tipo_histologico": calcular_taxa_acerto("acertou_tipo_histologico"),
    }

    print(f"\n✓ Processamento concluído: {total_laudos} laudos")
    display(df_respostas)

### Registro de métricas e experimentos no MLflow
**Objetivo da Célula:** Criar ou recuperar um experimento no MLflow e registrar as métricas de qualidade da extração.

**Dependências:**
- Variável `json_metricas` com resultados agregados da avaliação
- Bibliotecas mlflow e json

**Funções Definidas:**
- `get_or_create_experiment(experiment_name)`: Obtém ID de experimento existente ou cria um novo

**Lógica Detalhada:**
1. Define a função `get_or_create_experiment` para encontrar ou criar um experimento MLflow
2. Obtém o ID do experimento para saúde preventiva mamária
3. Configura o MLflow para registro automático de informações adicionais (`mlflow.autolog()`)
4. Define um threshold de qualidade (80%) para avaliar se as extrações são aceitáveis
5. Inicia uma nova execução (run) do MLflow
6. Para cada campo avaliado, registra:
   - Taxa de acerto (métrica principal)
   - Flag indicando se passou no threshold
   - Contagens absolutas de acertos e total
7. Registra o ID da execução para referência futura

**Parâmetros Registrados:**
- `modelo`: Nome do modelo LLM utilizado ("databricks-llama-4-maverick")

**Métricas Registradas:**
- Para cada campo (`descritores_malignidade`, `grau_histologico`, etc.):
  - `{campo}_taxa_acerto`: Porcentagem de extrações corretas
  - `{campo}_passou_threshold`: Flag binária (1 = passou, 0 = falhou)
  - `{campo}_acertos`: Número absoluto de extrações corretas
  - `{campo}_total`: Número total de documentos avaliados

**Saída/Impacto:**
- Cria um registro permanente da qualidade do modelo no sistema MLflow
- Permite comparação da performance entre diferentes versões do modelo
- Fornece ID de execução para referência e rastreabilidade

In [0]:
# import json
# import mlflow

# def get_or_create_experiment(experiment_name):
#     experiment = mlflow.get_experiment_by_name(experiment_name)
#     if experiment:
#         experiment_id = experiment.experiment_id
#     else:
#         experiment_id = mlflow.create_experiment(experiment_name)
#     mlflow.set_experiment(experiment_name)
#     return experiment_id

# # Registrar métricas no MLflow apenas se houver dados processados
# if df_spk.count() > 0:
#     experiment_id = get_or_create_experiment("/Shared/saude_preventiva_mama/experiments_fleury_anatomopatologico")
#     mlflow.autolog()

# threshold = 0.8

# with mlflow.start_run(experiment_id=experiment_id) as run:
#     mlflow.log_param("modelo", ENDPOINT_NAME)
#     mlflow.log_param("total_laudos", total_laudos)
    
#     for campo, stats in json_metricas.items():
#         taxa = stats["taxa_acerto"]
#         mlflow.log_metric(f"{campo}_taxa_acerto", taxa)
#         passou_flag = 1 if taxa >= threshold else 0
#         mlflow.log_metric(f"{campo}_passou_threshold", passou_flag)
#         mlflow.log_metric(f"{campo}_acertos", stats["acertos"])
#         mlflow.log_metric(f"{campo}_total", stats["total"])
    
#     run_id = mlflow.active_run().info.run_id
#     print(f"Run registrada: {run_id}")

### Visualização das métricas de qualidade da extração
**Objetivo da Célula:** Exibir as métricas de qualidade da extração para análise visual.

Esta célula usa a função `display()` para mostrar o dicionário de métricas `json_metricas` que foi calculado anteriormente. Estas métricas fornecem uma visão detalhada do desempenho da extração de informações pelo modelo LLM em comparação com a extração por RegEx, incluindo taxas de acerto para cada campo extraído.

In [0]:
display(json_metricas)

### Persistência dos dados na tabela Delta
**Objetivo da Célula:** Salvar os dados processados na tabela Delta de destino usando uma estratégia de merge.

**Dependências:**
- DataFrame `df_respostas` contendo os dados processados
- Biblioteca Delta para operações de merge em tabelas
- Octoops (Sentinel) para monitoramento e alertas

**Funções Definidas:**
- `insert_data(df_spk, output_data_path)`: Realiza o merge dos dados na tabela Delta especificada

**Lógica Detalhada:**
1. Define a constante `WEBHOOK_DS_AI_BUSINESS_STG` como 'stg' (ambiente de staging)
2. Define a constante `OUTPUT_DATA_PATH` com o nome da tabela de destino
3. Implementa a função `insert_data`:
   - Carrega a tabela Delta existente
   - Realiza um merge usando o DataFrame como origem
   - Utiliza chaves de junção: ficha, id_item e id_subitem
   - Atualiza registros existentes e insere novos registros
4. Em um bloco try-except:
   - Verifica se o DataFrame tem registros (`df_respostas.count() > 0`)
   - Em caso positivo, chama `insert_data` para persistir os dados
   - Em caso negativo, envia alerta via Sentinel informando ausência de laudos para extração
   - Em caso de exceção, captura o erro, imprime o traceback e relança a exceção

**Saída/Impacto:**
- Os dados processados são salvos na tabela Delta usando estratégia de merge
- O sistema registra a quantidade de registros salvos
- Em caso de falha ou ausência de dados, um alerta é enviado via Sentinel para monitoramento
- Exceções são devidamente registradas e relançadas para o sistema de monitoramento

In [0]:
# from octoops import Sentinel
# import traceback

# WEBHOOK_DS_AI_BUSINESS_STG = 'stg'
# OUTPUT_DATA_PATH = "refined.saude_preventiva.fleury_laudos_mama_anatomia_patologica_v2"

# # Função para salvar dados na tabela Delta
# def insert_data(df_spk, output_data_path):  
#     # Carrega a tabela Delta existente
#     delta_table = DeltaTable.forName(spark, output_data_path)

#     # Faz o merge (upsert)
#     (delta_table.alias("target")
#         .merge(
#             df_spk.alias("source"),
#             "target.ficha = source.ficha AND target.id_item = source.id_item AND target.id_subitem = source.id_subitem"
#         )
#         .whenMatchedUpdateAll()  # atualiza todos os campos se o ID já existir
#         .whenNotMatchedInsertAll()  # insere se o ID não existir
#         .execute())

# try:
#     if df_spk.count() > 0:        
#         insert_data(df_respostas, OUTPUT_DATA_PATH)
#         print('Total de registros salvos na tabela:', df_respostas.count())
#     else: 
#         error_message = "Fleury AnatomoPatologico - Não há laudos para extração."
#         sentinela_ds_ai_business = Sentinel(
#             project_name='Monitor_Linhas_Cuidado_Mama',
#             env_type=WEBHOOK_DS_AI_BUSINESS_STG,
#             task_title='Fleury AnatomoPatologico'
#         )

#         sentinela_ds_ai_business.alerta_sentinela(
#             categoria='Alerta', 
#             mensagem=error_message,
#             job_id_descritivo='3_fleury_mama_anatomopatologico'
#         )
# except Exception as e:
#     traceback.print_exc()
#     raise e

In [0]:
df2 = df_respostas.toPandas()
salvar_excel(df2, "resultados_anatomo_medicos_5.xlsx")